In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
import calendar
#import prophet
from datetime import datetime
import holidays


warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

plt.rcParams["figure.figsize"] = (10,8)

In [2]:
df = pd.read_csv("../data/bikes.csv", index_col = 0 )

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 97.0+ KB


In [4]:
df.sample(5)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
306,307,03-11-2018,winter,0,11,0,4,1,1,16.741653,20.29605,70.2083,9.166739,318,3656,3974
239,240,28-08-2018,autumn,0,8,0,0,0,1,28.989419,32.39795,56.1765,20.412153,1415,2919,4334
201,202,21-07-2018,autumn,0,7,0,4,1,2,33.415000,41.31855,69.1250,14.875407,632,3152,3784
468,469,14-04-2019,summer,1,4,0,6,0,1,20.295000,24.39980,50.2917,12.791439,2795,4665,7460
247,248,05-09-2018,autumn,0,9,1,1,0,2,27.606653,31.28230,79.0417,14.250632,1236,2115,3351


- 1. En la columna yr, el 0 hace referencia a 2018 y el 1 2019 ---> cambiar a 2018 y 2019
- 2. Revisar estaciones segun meses - DONE  
- 3. Los dias de la semana pasarlos a str (texto)
- 4. Redondear temperaturas
- 5. Cambiar dteday a tipo daytime - DONE 
- 6. Comprobar festivos - columna holiday - 16 días - DONE
- 7. Los dias de la semana no corresponden con los que debieran ser - DONE
- 9. Librería calendar - dia de la semana DONE.
- 10. Chequear funcion seasons - DONE

Weathersit :
- 1: Sol y nubes
- 2: Niebla - nublado
- 3: Regulinchi
- 4: Tiempo chungo

In [5]:
df.duplicated().sum()

0

In [6]:
df.shape

(730, 16)

---

### Reasignamos columna season

---

In [7]:
#Cambiar columna 'dteday' a datatime

df.dteday =pd.to_datetime(df.dteday, dayfirst=True)

In [8]:
primavera = df[df['season']=='spring'] #Primavera 20 marzo - 21 junio
primavera['mnth'].unique()

array([ 1,  2,  3, 12])

In [9]:
invierno = df[df['season']=='winter'] # Invierno 21 de diciembre - 20 de marzo
invierno['mnth'].unique()

array([ 9, 10, 11, 12])

In [10]:
#Verano 21 junio - 23 septiembre 
verano = df[df['season']=='summer'] 
verano['mnth'].unique()

array([3, 4, 5, 6])

In [11]:
#Otoño 23 septiembre - 21 de diciembre
otoño = df[df['season']=='autumn'] 
otoño['mnth'].unique()

array([6, 7, 8, 9])

In [12]:
mapa_estaciones ={"spring": "winter", "winter": "autumn", "summer":"spring", "autumn":"summer"}

In [13]:
df["season"] = df["season"].map(mapa_estaciones)

In [14]:
#Primavera 21 marzo - 20 junio
#Verano 21 junio - 22 septiembre 
#Otoño 23 septiembre - 20 de diciembre
#Invierno 21 de diciembre - 20 de marzo

--- 

### Ajustamos los días de la semana

---

In [15]:
def dia_semana(col):
    return col.isoweekday()

In [16]:
df["weekday"]= df["dteday"].apply(dia_semana)

---

### Ajustamos columna holiday

---

In [17]:
df.workingday.value_counts()

1    499
0    231
Name: workingday, dtype: int64

In [18]:
df.holiday.value_counts()

0    709
1     21
Name: holiday, dtype: int64

In [19]:
#Sacamos ls festivos con la libreria 
from datetime import date
import holidays
  
us_holidays = holidays.US()
festivos_nuevo = []
for i in holidays.US(years = [2018, 2019], state = 'WA').items():
    festivos_nuevo.append(i)


In [20]:
#Creamos una lista con los festivos que faltan de la lista proporcionada por la empresa 
festivos_añadir = ['2018-02-13', '2018-03-31', '2018-07-05', '2018-07-28', '2018-11-24', '2018-12-24','2018-12-31', '2019-11-29', '2019-02-13' , '2019-03-31', '2019-07-05'
,'2019-07-28' , '2019-12-24' , '2019-12-31']

In [21]:
#Apendeamos la lista
in_holidays = holidays.HolidayBase()
in_holidays.append(festivos_añadir)

In [22]:
#Convertimos a diccionario para poder añadirlo a las vacaciones que teniamos 
dict_holidays = dict(festivos_nuevo)

In [23]:
in_holidays.update(dict_holidays)

In [24]:
lista_festivos = list(in_holidays.keys())

In [25]:
len(lista_festivos)

35

In [26]:
print(lista_festivos)

[datetime.date(2018, 2, 13), datetime.date(2018, 3, 31), datetime.date(2018, 7, 5), datetime.date(2018, 7, 28), datetime.date(2018, 11, 24), datetime.date(2018, 12, 24), datetime.date(2018, 12, 31), datetime.date(2019, 11, 29), datetime.date(2019, 2, 13), datetime.date(2019, 3, 31), datetime.date(2019, 7, 5), datetime.date(2019, 7, 28), datetime.date(2019, 12, 24), datetime.date(2019, 12, 31), datetime.date(2018, 1, 1), datetime.date(2018, 1, 15), datetime.date(2018, 2, 19), datetime.date(2018, 5, 28), datetime.date(2018, 7, 4), datetime.date(2018, 9, 3), datetime.date(2018, 10, 8), datetime.date(2018, 11, 11), datetime.date(2018, 11, 12), datetime.date(2018, 11, 22), datetime.date(2018, 12, 25), datetime.date(2019, 1, 1), datetime.date(2019, 1, 21), datetime.date(2019, 2, 18), datetime.date(2019, 5, 27), datetime.date(2019, 7, 4), datetime.date(2019, 9, 2), datetime.date(2019, 10, 14), datetime.date(2019, 11, 11), datetime.date(2019, 11, 28), datetime.date(2019, 12, 25)]


In [27]:
df["holiday"] = df["dteday"].isin(lista_festivos)
df.sample(3)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
609,610,2019-09-02,summer,1,9,True,1,0,2,28.563347,32.45000,81.5000,4.292744,2613,3197,5810
79,80,2018-03-21,spring,0,3,False,3,1,2,17.647835,20.48675,73.7391,19.348461,401,1676,2077
440,441,2019-03-17,winter,1,3,False,7,0,2,21.080847,25.25230,75.5833,7.417168,3155,4681,7836


In [28]:
df.loc[df['holiday'] == False, 'holiday'] = 0
df.loc[df['holiday'] == True, 'holiday'] = 1

In [29]:
df.holiday.value_counts()

0    695
1     35
Name: holiday, dtype: int64

---

### Ajustamos columna workingday

---

In [30]:
def workingday (col_holiday, col_weekday):
    if col_holiday == 1:
        return 'NO'
    elif col_weekday == 6 or col_weekday == 7:
        return 'NO'
    else:
        return 'SI'

In [31]:
df['workingday'] = df.apply(lambda brocoli: workingday(brocoli['holiday'], brocoli['weekday']), axis = 1)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   instant     730 non-null    int64         
 1   dteday      730 non-null    datetime64[ns]
 2   season      730 non-null    object        
 3   yr          730 non-null    int64         
 4   mnth        730 non-null    int64         
 5   holiday     730 non-null    object        
 6   weekday     730 non-null    int64         
 7   workingday  730 non-null    object        
 8   weathersit  730 non-null    int64         
 9   temp        730 non-null    float64       
 10  atemp       730 non-null    float64       
 11  hum         730 non-null    float64       
 12  windspeed   730 non-null    float64       
 13  casual      730 non-null    int64         
 14  registered  730 non-null    int64         
 15  cnt         730 non-null    int64         
dtypes: datetime64[ns](1), floa

_____

Apuntes sprint1:
- Graficas análisis exploratorio de cara a la demo
- Cuidado con las variables predictoras -  habrá que probar diferentes cosas
- Variables categóricas -  sacarlas de los boxplot - para poder analizar a part
- Si no cumplimos normalidad - Random forest y fuera
- Idiomas para la demo
- Conteo fuera de nuestro modelo predictivo

--- 
### Homogeneizamos las columnas a castellano

---

In [33]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2018-01-01,winter,0,1,1,1,NO,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,2018-01-02,winter,0,1,0,2,SI,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,2018-01-03,winter,0,1,0,3,SI,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,2018-01-04,winter,0,1,0,4,SI,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,2018-01-05,winter,0,1,0,5,SI,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [34]:
df.columns.to_list

<bound method IndexOpsMixin.tolist of Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')>

In [35]:
nombres_nuevos = {'instant': "registro", 'dteday':"fecha", 'season':"estacion", 'yr':"año", 'mnth':"mes", 'holiday':"vacaciones", 'weekday':"dia_semana",
                    'workingday':"laborable", 'weathersit':"clima", 'temp':"temperatura", 'atemp':"sensacion_termica", 'hum':"humedad", 'windspeed':"viento",
                     'casual':"casuales", 'registered':"registrados", 'cnt':"totales"}

In [36]:
df.rename(columns=nombres_nuevos, inplace=True)

In [37]:
df.head(2)

,registro,fecha,estacion,año,mes,vacaciones,dia_semana,laborable,clima,temperatura,sensacion_termica,humedad,viento,casuales,registrados,totales
0,1,2018-01-01,winter,0,1,1,1,NO,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,2018-01-02,winter,0,1,0,2,SI,2,14.902598,17.68695,69.6087,16.652113,131,670,801


In [38]:
#Guardamos los cambios realizados
df.to_csv('../data/01_bikes_limpieza.csv')

---

## Separamos los df en casuales y regristrados

---

In [39]:
df_registrados = df.drop(["registro","totales", "casuales"], axis=1 )

In [40]:
df_casuales = df.drop(["registro", "totales", "registrados"], axis=1)

In [41]:
df_registrados.to_csv('../data/01_bikes_registrados.csv')

In [42]:
df_casuales.to_csv('../data/01_bikes_casuales.csv')